In [23]:
import pandas as pd
import numpy as np
import lightgbm as lgb
from sklearn.model_selection import StratifiedKFold,GridSearchCV
from sklearn.model_selection import cross_val_score, learning_curve
from sklearn.metrics import mean_absolute_error, make_scorer
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
import matplotlib.pyplot as plt
import pickle
import hyperopt.pyll.stochastic
import time

In [24]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
test['loss'] = np.nan
joined = pd.concat([train, test])

In [25]:
features = [x for x in train.columns if x not in ['id','loss', 'log_loss']]

cat_features = [x for x in train.select_dtypes(
        include=['object']).columns if x not in ['id','loss', 'log_loss']]
num_features = [x for x in train.select_dtypes(
        exclude=['object']).columns if x not in ['id','loss', 'log_loss']]

print "Categorical features:", len(cat_features)
print "Numerical features:", len(num_features)

Categorical features: 116
Numerical features: 14


In [26]:
def evalerror(preds, dtrain):
    labels = dtrain.get_label()
    return 'mae', mean_absolute_error(np.exp(preds), np.exp(labels)),False

def xg_eval_mae(yhat, dtrain):
    y = dtrain.get_label()
    return 'mae', mean_absolute_error(np.exp(y), np.exp(yhat)),False

def mae_score(y_true, y_pred):
    return mean_absolute_error(np.exp(y_true), np.exp(y_pred))

mae_scorer = make_scorer(mae_score, greater_is_better=False)

In [27]:
for column in list(train.select_dtypes(include=['object']).columns):
    if train[column].nunique() != test[column].nunique():
        set_train = set(train[column].unique())
        set_test = set(test[column].unique())
        remove_train = set_train - set_test
        remove_test = set_test - set_train

        remove = (remove_train|remove_test)
        def filter_cat(x):
            if x in remove:
                return np.nan
            return x

        joined[column] = joined[column].apply(lambda x: 'np.nan' if x in remove else x, 1)

    joined[column] = pd.factorize(joined[column].values, sort=True,na_sentinel=-1)[0]
    joined[column] = joined[column] + 1

train = joined[joined['loss'].notnull()]
test = joined[joined['loss'].isnull()]

shift = 200
y = np.log(train['loss'] + shift)
ids = test['id']
X = train.drop(['loss', 'id'], 1)
X_test = test.drop(['loss', 'id'], 1)
lgtrain = lgb.Dataset(X, label=y)
lgtest = lgb.Dataset(X_test)

In [28]:
time0= time.time()
space4lgb = {
    'max_depth': hp.choice('max_depth', range(3,29)),
    'min_child_weight':hp.choice('min_child_weight',range(2,100)),
    'num_leaves':hp.choice('num_leaves',range(50,400)),
    
    'min_split_gain':hp.choice('min_split_gain',[ 0.01 * i for i in range(0,100)]),
    'colsample_bytree':hp.choice('colsample_bytree',[ 0.01 * i for i in range(50,100)]),
    'subsample':hp.choice('subsample',[ 0.01 * i for i in range(50,100)]),
    
    'subsample_freq':hp.choice('subsample_freq',range(0,10)),
    
    'reg_alpha':hp.uniform('reg_alpha',0,1000),
    'reg_lambda':hp.uniform('reg_lambda',0,1000),
    
    'nthread': hp.choice('nthread', [10]),
}
minacc = 1512.60244356
def optf(params): 
    cv_result_lgb = lgb.cv(params, lgtrain, num_boost_round=100, nfold=3, 
                           stratified=False,
                    feval=evalerror, early_stopping_rounds=5) 
    acc = cv_result_lgb['mae-mean'][-1]
    print acc
    global minacc
    if acc < minacc:
        minacc = acc
        print 'new best:', minacc, params
        
        with open("lgbparams.txt", "a") as text_file:
            text_file.write('socre:{}, params:{}\n'.format(minacc,params))
    return {'loss': -acc, 'status': STATUS_OK}

In [29]:
def run_trials():

    trials_step = 1  # how many additional trials to do after loading saved trials. 1 = save after iteration
    max_trials = 1  # initial max_trials. put something small to not have to wait

    
    try:  # try to load an already saved trials object, and increase the max
        trials = pickle.load(open("/home/phe002/shicheng/dataset/lgbhyperopt", "rb"))
        max_trials = len(trials.trials) + trials_step
        print("Rerunning from {} trials to {} (+{}) trials".format(len(trials.trials), max_trials, trials_step))
    except:  # create a new trials object and start searching
        trials = Trials()

    best = fmin(fn=optf, space=space4lgb, algo=tpe.suggest, max_evals=max_trials, trials=trials)
    # save the trials object
    with open("/home/phe002/shicheng/dataset/lgbhyperopt", "wb") as f:
        pickle.dump(trials, f)

# loop indefinitely and stop whenever you like
while True:
    run_trials()

1306.962409
new best: 1306.962409 {'num_leaves': 295, 'reg_alpha': 854.0021532947828, 'subsample_freq': 5, 'colsample_bytree': 0.72, 'verbose': 1, 'nthread': 10, 'min_child_weight': 70, 'min_split_gain': 0.96, 'subsample': 0.5700000000000001, 'reg_lambda': 578.170473144676, 'max_bin': 255, 'max_depth': 21}
Rerunning from 1 trials to 2 (+1) trials
1209.46384034
new best: 1209.46384034 {'num_leaves': 399, 'reg_alpha': 152.1685142353606, 'subsample_freq': 8, 'colsample_bytree': 0.58, 'verbose': 1, 'nthread': 10, 'min_child_weight': 13, 'min_split_gain': 0.93, 'subsample': 0.7000000000000001, 'reg_lambda': 712.2930204028966, 'max_depth': 12}
Rerunning from 2 trials to 3 (+1) trials
1242.13282155
Rerunning from 3 trials to 4 (+1) trials
1229.60515599
Rerunning from 4 trials to 5 (+1) trials
1205.92945829
new best: 1205.92945829 {'num_leaves': 395, 'reg_alpha': 186.7479760162719, 'subsample_freq': 0, 'colsample_bytree': 0.6900000000000001, 'verbose': 1, 'nthread': 10, 'min_child_weight': 98,

1293.38072526
Rerunning from 120 trials to 121 (+1) trials
1308.77260893
Rerunning from 121 trials to 122 (+1) trials
1271.91293939
Rerunning from 122 trials to 123 (+1) trials
1164.26355925
new best: 1164.26355925 {'num_leaves': 76, 'reg_alpha': 1.770215931141081, 'subsample_freq': 2, 'colsample_bytree': 0.71, 'verbose': 1, 'nthread': 10, 'min_child_weight': 5, 'min_split_gain': 0.58, 'subsample': 0.85, 'reg_lambda': 526.1934474867576, 'max_depth': 20}
Rerunning from 123 trials to 124 (+1) trials
1241.50365096
Rerunning from 124 trials to 125 (+1) trials
1298.9988851
Rerunning from 125 trials to 126 (+1) trials
1275.75523482
Rerunning from 126 trials to 127 (+1) trials
1243.7108426
Rerunning from 127 trials to 128 (+1) trials
1267.24486496
Rerunning from 128 trials to 129 (+1) trials
1279.58810249
Rerunning from 129 trials to 130 (+1) trials
1302.33454516
Rerunning from 130 trials to 131 (+1) trials
1266.21806858
Rerunning from 131 trials to 132 (+1) trials
1275.05962715
Rerunning fro

1259.96087282
Rerunning from 255 trials to 256 (+1) trials
1267.55622817
Rerunning from 256 trials to 257 (+1) trials
1306.29051371
Rerunning from 257 trials to 258 (+1) trials
1325.96546186
Rerunning from 258 trials to 259 (+1) trials
1304.67091297
Rerunning from 259 trials to 260 (+1) trials
1307.08206857
Rerunning from 260 trials to 261 (+1) trials
1300.39506905
Rerunning from 261 trials to 262 (+1) trials
1338.48956872
Rerunning from 262 trials to 263 (+1) trials
1337.68900651
Rerunning from 263 trials to 264 (+1) trials
1290.50210698
Rerunning from 264 trials to 265 (+1) trials
1326.37238916
Rerunning from 265 trials to 266 (+1) trials
1327.99985989
Rerunning from 266 trials to 267 (+1) trials
1337.134942
Rerunning from 267 trials to 268 (+1) trials
1308.55269548
Rerunning from 268 trials to 269 (+1) trials
1338.52332382
Rerunning from 269 trials to 270 (+1) trials
1332.88307102
Rerunning from 270 trials to 271 (+1) trials
1332.77067437
Rerunning from 271 trials to 272 (+1) trials

Rerunning from 394 trials to 395 (+1) trials
1263.2198723
Rerunning from 395 trials to 396 (+1) trials
1273.47652556
Rerunning from 396 trials to 397 (+1) trials
1288.92303729
Rerunning from 397 trials to 398 (+1) trials
1327.91125427
Rerunning from 398 trials to 399 (+1) trials
1257.72020816
Rerunning from 399 trials to 400 (+1) trials
1286.65664692
Rerunning from 400 trials to 401 (+1) trials
1296.34755624
Rerunning from 401 trials to 402 (+1) trials
1337.36726734
Rerunning from 402 trials to 403 (+1) trials
1337.78428659
Rerunning from 403 trials to 404 (+1) trials
1331.79015304
Rerunning from 404 trials to 405 (+1) trials
1336.99316099
Rerunning from 405 trials to 406 (+1) trials
1302.48827674
Rerunning from 406 trials to 407 (+1) trials
1332.24477388
Rerunning from 407 trials to 408 (+1) trials
1339.15479385
Rerunning from 408 trials to 409 (+1) trials
1323.12274398
Rerunning from 409 trials to 410 (+1) trials
1290.98496283
Rerunning from 410 trials to 411 (+1) trials
1287.6304379

1273.47214545
Rerunning from 529 trials to 530 (+1) trials
1292.66722653
Rerunning from 530 trials to 531 (+1) trials
1250.34077301
Rerunning from 531 trials to 532 (+1) trials
1331.52081214
Rerunning from 532 trials to 533 (+1) trials
1282.08171557
Rerunning from 533 trials to 534 (+1) trials
1285.17692712
Rerunning from 534 trials to 535 (+1) trials
1229.52476825
Rerunning from 535 trials to 536 (+1) trials
1312.7945225
Rerunning from 536 trials to 537 (+1) trials
1268.88494231
Rerunning from 537 trials to 538 (+1) trials
1325.49480188
Rerunning from 538 trials to 539 (+1) trials
1212.12324234
Rerunning from 539 trials to 540 (+1) trials
1258.50832229
Rerunning from 540 trials to 541 (+1) trials
1292.05285325
Rerunning from 541 trials to 542 (+1) trials
1306.211177
Rerunning from 542 trials to 543 (+1) trials
1218.62589639
Rerunning from 543 trials to 544 (+1) trials
1304.97431235
Rerunning from 544 trials to 545 (+1) trials
1311.68340616
Rerunning from 545 trials to 546 (+1) trials


...

1316.05037919
Rerunning from 3380 trials to 3381 (+1) trials
1297.48655771
Rerunning from 3381 trials to 3382 (+1) trials
1273.68449154
Rerunning from 3382 trials to 3383 (+1) trials
1284.17025821
Rerunning from 3383 trials to 3384 (+1) trials
1307.58972789
Rerunning from 3384 trials to 3385 (+1) trials
1287.69943125
Rerunning from 3385 trials to 3386 (+1) trials
1338.67571186
Rerunning from 3386 trials to 3387 (+1) trials
1311.22466065
Rerunning from 3387 trials to 3388 (+1) trials
1311.01999609
Rerunning from 3388 trials to 3389 (+1) trials
1329.80713544
Rerunning from 3389 trials to 3390 (+1) trials
1272.08458316
Rerunning from 3390 trials to 3391 (+1) trials
1329.17040595
Rerunning from 3391 trials to 3392 (+1) trials
1324.26395799
Rerunning from 3392 trials to 3393 (+1) trials
1236.08673812
Rerunning from 3393 trials to 3394 (+1) trials
1321.05637804
Rerunning from 3394 trials to 3395 (+1) trials
1312.64312306
Rerunning from 3395 trials to 3396 (+1) trials
1319.09354801
Rerunning 

1292.29090963
Rerunning from 3515 trials to 3516 (+1) trials
1302.14094173
Rerunning from 3516 trials to 3517 (+1) trials
1315.06176114
Rerunning from 3517 trials to 3518 (+1) trials
1303.3539129
Rerunning from 3518 trials to 3519 (+1) trials
1241.12652586
Rerunning from 3519 trials to 3520 (+1) trials
1295.09668555
Rerunning from 3520 trials to 3521 (+1) trials
1335.5760815
Rerunning from 3521 trials to 3522 (+1) trials
1331.01940049
Rerunning from 3522 trials to 3523 (+1) trials
1295.83950789
Rerunning from 3523 trials to 3524 (+1) trials
1283.54764892
Rerunning from 3524 trials to 3525 (+1) trials
1319.41984532
Rerunning from 3525 trials to 3526 (+1) trials
1324.22857148
Rerunning from 3526 trials to 3527 (+1) trials
1335.30932752
Rerunning from 3527 trials to 3528 (+1) trials
1292.04622339
Rerunning from 3528 trials to 3529 (+1) trials
1315.46167998
Rerunning from 3529 trials to 3530 (+1) trials
1316.69359059
Rerunning from 3530 trials to 3531 (+1) trials
1317.24964073
Rerunning fr

1303.15765298
Rerunning from 3650 trials to 3651 (+1) trials
1295.49686003
Rerunning from 3651 trials to 3652 (+1) trials
1298.27195381
Rerunning from 3652 trials to 3653 (+1) trials
1298.18240316
Rerunning from 3653 trials to 3654 (+1) trials
1298.16954331
Rerunning from 3654 trials to 3655 (+1) trials
1317.32766989
Rerunning from 3655 trials to 3656 (+1) trials
1334.52596429
Rerunning from 3656 trials to 3657 (+1) trials
1331.37892792
Rerunning from 3657 trials to 3658 (+1) trials
1291.94763101
Rerunning from 3658 trials to 3659 (+1) trials
1315.08960904
Rerunning from 3659 trials to 3660 (+1) trials
1339.01855905
Rerunning from 3660 trials to 3661 (+1) trials
1290.99837001
Rerunning from 3661 trials to 3662 (+1) trials
1287.41826907
Rerunning from 3662 trials to 3663 (+1) trials
1276.25613411
Rerunning from 3663 trials to 3664 (+1) trials
1330.00952467
Rerunning from 3664 trials to 3665 (+1) trials
1344.10451007
Rerunning from 3665 trials to 3666 (+1) trials
1303.4760524
Rerunning f

1342.26392776
Rerunning from 3785 trials to 3786 (+1) trials
1305.97079921
Rerunning from 3786 trials to 3787 (+1) trials
1312.13130258
Rerunning from 3787 trials to 3788 (+1) trials
1286.09867657
Rerunning from 3788 trials to 3789 (+1) trials
1286.4175716
Rerunning from 3789 trials to 3790 (+1) trials
1344.18108308
Rerunning from 3790 trials to 3791 (+1) trials
1320.57905867
Rerunning from 3791 trials to 3792 (+1) trials
1297.50407149
Rerunning from 3792 trials to 3793 (+1) trials
1289.3066806
Rerunning from 3793 trials to 3794 (+1) trials
1275.83568498
Rerunning from 3794 trials to 3795 (+1) trials
1301.69251277
Rerunning from 3795 trials to 3796 (+1) trials
1307.55127571
Rerunning from 3796 trials to 3797 (+1) trials
1345.84784477
Rerunning from 3797 trials to 3798 (+1) trials
1305.43040577
Rerunning from 3798 trials to 3799 (+1) trials
1294.76886066
Rerunning from 3799 trials to 3800 (+1) trials
1304.0068067
Rerunning from 3800 trials to 3801 (+1) trials
1276.3105865
Rerunning from

KeyboardInterrupt: 

In [66]:
'''params = {
    'min_child_weight': 1,
    'eta': 0.01,
    'colsample_bytree': 0.5,
    'max_depth': 12,
    'subsample': 0.8,
    'alpha': 1,
    'gamma': 1,
    'silent': 1,
    'verbose_eval': True,
    'seed': RANDOM_STATE,
    'nthread':10
}

xgtrain = xgb.DMatrix(X, label=y)
xgtest = xgb.DMatrix(X_test)

model = xgb.train(params, xgtrain, int(2012 / 0.9), feval=evalerror)'''

In [ ]:
prediction = np.exp(model.predict(xgtest)) - shift
submission = pd.DataFrame()
submission['loss'] = prediction
submission['id'] = ids
submission.to_csv('sub_v.csv', index=False)

In [ ]:
if __name__ == '__main__':
    print "a+b"